In [3]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import re
import csv
import numpy as np
from copy import deepcopy

"""
Function used in sort
"""


def takeFirst(list):
    return int(list[0])


"""

This is the function that used to write the result to a new CSV file

"""


def writeCSV(resultList):
    with open('result.csv', 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(["Id", "Class"])
        for list in resultList:
            csv_writer.writerow(list)
        csv_file.close()


"""
This function is used to generate two features dictionary, username dictionary and location dictionary.
It will remove some low frequency features according to the number given in the parameter
 

"""


def featureEng(data, locNumber, userNumber):
    # read tsv file
    csv.register_dialect('mydialect', delimiter='\t', quoting=csv.QUOTE_ALL)

    locDict = {}
    userDict = {}
    csvfile = open(data, "r")
    reader = csv.reader(csvfile, 'mydialect')

    # read tsv file line by line
    for row in reader:
        # replace special character
        tweeteStr = row[2].replace(',', ' ').replace('/', ' ').replace('\\', ' ').replace(')', ' ').replace('.', ' ')

        # use reagular expression to find all the location string(format @ location ) and user name string(format @username )
        locationList = re.findall(r"@ (.*)", tweeteStr)
        userList = re.findall(r"@(.+?) ", tweeteStr)

        # add them to the dictionary
        if len(locationList) > 0:
            locationList = locationList[0].split()
            for feature in locationList:
                if feature.istitle() and feature not in locDict:
                    locDict[feature] = 0
                elif feature in locDict:
                    locDict[feature] += 1

        if len(userList) > 0:
            for feature in userList:
                if feature not in userDict:
                    userDict[feature] = 0
                elif feature in userDict:
                    userDict[feature] += 1

    # reomve the low frequency features in the location dictionary
    LocDelList = []
    for item in locDict:
        if locDict[item] <= locNumber or len(item) == 1:
            LocDelList.append(item)

    for item in LocDelList:
        locDict.pop(item)

    for item in locDict:
        locDict[item] = 0

    # reomve the low frequency features in the user dictionary
    UserdelList = []
    for item in userDict:
        if userDict[item] <= userNumber or len(item) == 1:
            UserdelList.append(item)

    for item in UserdelList:
        userDict.pop(item)

    for item in userDict:
        userDict[item] = 0

    csvfile.close()
    return locDict, userDict


"""
This function is used to compute the number of the features in each single sample
It will return a array that record the count of each features in all the samples(the sample's ID must in the IDlist)
 
"""

"""
By the way. I CANT UNDERSTAND WHY TRAIN_RAW。TSV ID IS DIFFERENT FROM TRAIN_TOP100.CSV
IT WASTE ME TOO MUCH TIME ON FINDING THE BUG

"""

def computeFeatures(tsv, adict, IDlist, trainset ):
    vectors = []
    f = open(tsv, "r")
    # read each row

    if trainset == True:
        for row in f:
            row = row.split("\t")
            if str(row[0]) in IDlist:
                rowDict = deepcopy(adict)
                # row[2] is the tweet text
                data = str(row[2])
                data = data.replace(',', ' ').replace('/', ' ').replace('\\', ' ').replace(')', ' ').replace('.', ' ')

                # count the number of the features and save to the dictionary
                for word in rowDict:
                    if word in data:
                        rowDict[word] += 1
                vectors.append(list(rowDict.values()))

        return np.array(vectors)

    else:
        for row in f:
            row = row.split("\t")
            if str(row[0][1:]) in IDlist:
                rowDict = deepcopy(adict)
                # row[2] is the tweet text
                data = str(row[2])
                data = data.replace(',', ' ').replace('/', ' ').replace('\\', ' ').replace(')', ' ').replace('.', ' ')

                # count the number of the features and save to the dictionary
                for word in rowDict:
                    if word in data:
                        rowDict[word] += 1
                vectors.append(list(rowDict.values()))

        return np.array(vectors)



"""
This function is used to import the preprocessed top100 features that provided by the assignment
It will return a array that contain these features and a label list and a Id list

"""


def importFeatures(data):
    vectors = []
    label = []
    idList = []
    csvfile = open(data, "r")
    reader = csv.reader(csvfile)
    for row in reader:
        vectors.append(row[1:-2])
        label.append(str(row[-1]))
        idList.append(str(row[0][1:]))

    for i in range(len(vectors)):
        for j in range(len(vectors[i])):
            vectors[i][j] = float(vectors[i][j])

    csvfile.close()

    return np.array(vectors), label, idList


"""

This is the function is a different import. It will output (all-0 data and their labels) and (non-all-0 data and their labels)
output 1.a array of the vectors of the features of the non-0 data 2.a list of label of these non-0 data 3.ID of these non-0 data
4.ID of the all-0 data


"""


def seperateImport(data):
    vectors = []
    label=[]
    idList = []
    emptyID = []
    emptyLabel = []
    csvfile = open(data, "r")
    reader = csv.reader(csvfile)
    for row in reader:
        counter = 2
        for i in range(len(row)):
            if counter == len(row) - 1:
                emptyLabel.append(str(row[-1]))
                emptyID.append(str(row[0][1:]))
                break

            elif row[counter] != row[counter - 1]:
                vectors.append(row[1:-2])
                idList.append(str(row[0][1:]))
                label.append(str(row[-1]))
                break

            elif row[counter] == row[counter - 1]:
                counter += 1

    for i in range(len(vectors)):
        for j in range(len(vectors[i])):
            vectors[i][j] = float(vectors[i][j])

    csvfile.close()

    return np.array(vectors), idList,label, emptyID, emptyLabel


if __name__ == "__main__":
    """
    To make it easy to distinguish, In this part.I will call the non-all-0-data as part A data 
    all-0-data as part B data
    
    
    For the non-all-0 data.Since the preprocessed vectors is already very useful to predict them. I will just add a small
    amount of the location features in the existing data to make a high correction rate
    
    For the all-0 data.Since the preprocessed vectors are all 0. So I will use extra features to predict them.
    These features include: location features and @username features
    
                                                                  
    """

    # initialize different classifier
    output = []
    svc = SVC()
    knn = KNeighborsClassifier()
    nb = GaussianNB()
    sgd = SGDClassifier()
    RF = RandomForestClassifier(n_estimators=300, n_jobs=-1, oob_score=True)
    DTC = DecisionTreeClassifier()
    NNMLP = MLPClassifier()
    ABC = AdaBoostClassifier()
    QDA = QuadraticDiscriminantAnalysis()

    """
    Import training and testing data
    """
    givenVectors, trainLabel, trainID = importFeatures("train-top100.csv")
    testA_vectors, testA_id, testA_label, test_emptyID, test_emptyLabel = seperateImport("test-top100.csv")

    """
    The prediction for part A Data
    """
    # create location dictionary, 9999 means I dont want userDict,45 means I only need the location that appear > 45
    locDict, _ = featureEng("train-raw.tsv", 40, 9999)

    # mix given vectors and self-generate vectors -training

    selfVectors = computeFeatures("train-raw.tsv", locDict, trainID, True)
    finalVectors_A = np.hstack((givenVectors, selfVectors))

    #compute features in test set and mix with given vectors
    selfVectors_testA = computeFeatures("test-raw.tsv", locDict, testA_id, False )
    finalVectors_testA = np.hstack((testA_vectors, selfVectors_testA))


    # standardlize vectors
    scaler = preprocessing.StandardScaler().fit(finalVectors_A)
    scaler.transform(finalVectors_A)
    scaler.transform(finalVectors_testA)

    DTC.fit(finalVectors_A, trainLabel)
    DTC_pred = DTC.predict(finalVectors_testA)
    #print(accuracy_score(DTC_pred, testA_label))
    # write the part A result in to output list
    for i in range (len(DTC_pred)):
        output.append([testA_id[i],str(DTC_pred[i])])


    """
    The prediction for part B Data
    """
    #re-generate the dictionary. Reason: the all-0 data has very few meaningful data, thus we need more features to predict
    locDict, userDict = featureEng("train-raw.tsv", 0, 20)

    #compute the features for the training set
    locVectors = computeFeatures("train-raw.tsv", locDict, trainID,True)
    userVectors = computeFeatures("train-raw.tsv", userDict, trainID,True)
    finalVectors_B = np.hstack((locVectors, userVectors))

    # compute features in test set and mix with given vectors
    locVectors_testB = computeFeatures("test-raw.tsv", locDict, test_emptyID,False)
    userVectors_testB = computeFeatures("test-raw.tsv", userDict, test_emptyID,False)
    finalVectors_testB = np.hstack((locVectors_testB, userVectors_testB))

    # standardlize vectors
    scaler = preprocessing.StandardScaler().fit(finalVectors_B)
    scaler.transform(finalVectors_B)
    scaler.transform(finalVectors_testB)

    DTC = DecisionTreeClassifier()
    DTC.fit(finalVectors_B, trainLabel)
    DTC_pred = DTC.predict(finalVectors_testB)
    #print(accuracy_score(DTC_pred, test_emptyLabel))

    for i in range (len(DTC_pred)):
        output.append([test_emptyID[i],str(DTC_pred[i])])

    output.sort(key=takeFirst)
    for item in output:
        item[0]='3'+item[0]


    writeCSV(output)








